# <center> Implementing a Key-Value Database 
    
A **key-value database**, or **key-value store**, is a data storage paradign designed for storing, retrieving, and managing associative arrays, and a datastructure more commonly known today as a dictionary or hash table. 
    
The goal of this project is implementing a key-value database on B-Tree data structure. 

## Importing and Initializing 

In [1]:
from btree import BTree 

class KVStore(BTree): 
    
    def __init__(self): 
        super().__init__(2)

## Overriding the Add Method 

The goal is to override the add() method and implement it so that it does the following : 

- If the key was already added, then we get the node that the containst it and replace the value asscoated with it by the new value. The BTree already has a \_find\_node() method that allows us to find a node containing a given key(if one exists).

- Otherwise, we add the new entry normally using the add() method from the BTree. To specify that we want to call the add() method from the BTree and not the one currently defined, we need to call it using the super() function, like so:super().add()

In [2]:
from btree import BTree 

class KVStore(BTree): 
    
    def __init__(self): 
        super().__init__(2)
        
    def add(self, key, value): 
        node = self._find_node(self.root, key)
        if node is None : 
            super().add(key, value)
        else : 
            for i, node_key in enumerate(node.keys) : 
                if node_key == key:
                    node.values[i] = value 

### Testing Overriding 

In [3]:
kv = KVStore() 

# The split threshold of a KVStore equals 2
assert kv.split_threshold == 2, "The split is not eqaul to 2."

# Add entry to kv 
kv.add(2, 'value1')

# Retrievve value with key 2 
assert kv.get_value(2) == 'value1', 'The value of key 2 is value1'

# Overriding new entry to kv w
kv.add(2, 'value2') 
assert kv.get_value(2) == 'value2', 'The value of key 2 is value2.'

## Implementing the Item Getter and Setter 

In [4]:
from btree import BTree 

class KVStore(BTree): 
    
    def __init__(self): 
        super().__init__(2)
        
    def add(self, key, value): 
        node = self._find_node(self.root, key)
        if node is None : 
            super().add(key, value)
        else : 
            for i, node_key in enumerate(node.keys) : 
                if node_key == key:
                    node.values[i] = value 
                
    def __getitem__(self, key):
        return self.get_value(key)
    
    def __setitem__(self, key, value): 
        self.add(key, value)

### Testing Getter and Setter 

In [5]:
kv = KVStore() 

# The split threshold of a KVStore equals 2
assert kv.split_threshold == 2, "The split is not eqaul to 2."

# Add entry to kv 
kv[2] = 'value1'

# Retrievve value with key 2 
assert kv[2] == 'value1', 'The value of key 2 is value1'

# Overriding new entry to kv w
kv[2] = 'value2'
assert kv[2] == 'value2', 'The value of key 2 is value2.'

## Enhancing the Contains Method

To enable 'in' operator on a custom class, we need to implement the \_\_contains\_\_() method that chekcs whether a given key is contained in the data structure. 

In [6]:
from btree import BTree 

class KVStore(BTree): 
    
    def __init__(self): 
        super().__init__(2)
        
    def add(self, key, value): 
        node = self._find_node(self.root, key)
        if node is None : 
            super().add(key, value)
        else : 
            for i, node_key in enumerate(node.keys) : 
                if node_key == key:
                    node.values[i] = value 
                
    def __getitem__(self, key):
        return self.get_value(key)
    
    def __setitem__(self, key, value): 
        self.add(key, value)
        
    def __contains__(self, key):
        return self.contains(key)

### Testing Contains Method

In [7]:
kv = KVStore() 

# Add entry to kv 
kv['key1'] = 'value1'

# Retrievve whether key1 is in instance
assert 'key1' in kv, "The key is in kv"

## Range Queries 

In [8]:
from btree import BTree 

class KVStore(BTree): 
    
    def __init__(self): 
        super().__init__(2)
        
    def add(self, key, value): 
        node = self._find_node(self.root, key)
        if node is None : 
            super().add(key, value)
        else : 
            for i, key in enumerate(node.keys) : 
                node.values[i] = value 
                
    def __getitem__(self, key):
        return self.get_value(key)
    
    def __setitem__(self, key, value): 
        self.add(key, value)
        
    def __contains__(self, key):
        return self.contains(key)
    
    def _range_intersect(self, range_start, range_end, node_min, node_max):
        if not node_min and node_min > range_end : 
            return False
        if not node_max and node_max < ragne_start : 
            return False 
        return True 
    
    def _range_query(self, range_start, range_end, current_node, min_key, max_key):
        if not self._range_intersect(range_start, range_end, min_key, max_key) : 
            return []
        results = []
        for i, key in enumerate(current_node.keys):
            if range_start <= key and key <= range_end:
                results.append(current_node.values[i])
        if not current_node.is_leaf():
            for i, child in enumerate(current_node.children):
                new_min_key = current_node.keys[i - 1] if i > 0 else min_key
                new_max_key = current_node.keys[i] if i < len(current_node) else max_key
                results += self._range_query(range_start, range_end, child, new_min_key, new_max_key)
        return results 

    def range_query(self, range_start, range_end):
        return self._range_query(range_start, range_end, self.root, float('-inf'), float('inf'))

In [9]:
class DictKVStore(dict) : 
    
    def range_query(self, range_start, range_end):
        result = []
        for key in self.keys() : 
            if range_start <= key and key <= range_end : 
                result.append(self[key])
        return result

In [10]:
dict_kv = DictKVStore()
our_kv = KVStore()
for i in range(10):
    dict_kv[i] = i
    our_kv[i] = i

for range_start, range_end in [(1, 3), (4, 6), (1, 10), (5, 5)]:
    dict_res = sorted(dict_kv.range_query(range_start, range_end))
    our_res = sorted(our_kv.range_query(range_start, range_end))
    assert dict_res == our_res, "Both data structures return the same range query result."

## Random Test 

Below is an implementation of a key-value store that uses a dictionary as the base data structure. We will perform the same operations on both and see if both provide the same results.

In [11]:
import random 

dict_kv = DictKVStore() 
kv = KVStore() 

print("Do Random Insertion")
for _ in range(20) : 
    key = random.randint(0, 100)
    value = random.randint(0, 1000)
    dict_kv[key] = value 
    kv[key] = value 
print("Random Inserting is Done. Random Testing will be start.")
print("===============================\n")

print("Testing Length")
assert len(dict_kv) == len(kv), f"Length of key-value store should be {len(dict_kv)}"

print("Testing values")
for key in dict_kv : 
    assert dict_kv[key] == kv[key], f"Wrong value for key {key}. Value should be {dict_kv[key]}" 
    
print("Testing Cotains")
for _ in range(20) : 
    key = random.randint(0, 200)  
    assert (key in dict_kv) == (key in kv), f"Wrong key {key} is or is not in kv."
    
print("Testing Range Queries")
for _ in range(20) : 
    range_start = random.randint(0, 80)
    range_end = random.randint(range_start, 160)
    dict_res = sorted(dict_kv.range_query(range_start, range_end)) 
    kv_res = sorted(kv.range_query(range_start, range_end))
    assert dict_res == kv_res, "Both data structure return the same range query result."

Do Random Insertion
Random Inserting is Done. Random Testing will be start.

Testing Length
Testing values
Testing Cotains
Testing Range Queries


## Performance Testing 

There are several ways we can compare the performance of both data structures. The overall idea is as follows : 

1. Generate many entries (about 50,000 should be enough) 
2. Generate many query interval. It's better to generate in a way that increases the number of query results from 1 to some given value. This way, we can see the impact of the number of results in the performance gained.
3. For each entry compute the ratio between the runtime using the dictionary based implementation and the B-tree-based implementation. A ration of 1 means that both took the same time, a ratio of 2 means that the B-Tree implementaion was two times fasetr, and so on.
4. Plot the runtime ratios to better visualize the results. 

The 'entries.csv' file contains a list of 50,000 entries, and the 'queries.csv' file contains a list of 1,000 queries with results range from 1 to 1,000. 

In [25]:
import csv 
import time 

dict_kv = DictKVStore()
kv = KVStore() 

# Call entries.csv and add entries into dict_kv, kv instance 

with open('entries.csv', 'r') as file : 
    reader = csv.reader(file) 
    rows = list(reader)[1:]
    for row in rows : 
        key = int(row[0])
        value = int(row[1])
        dict_kv[key] = value 
        kv[key] = value 
        
# Call queries.csv and execute range_query() method with range_start and range_end 
# And then calculate execution time ratio of DictKVStore and KVStore 
        
time_ratios = []
with open('queries.csv', 'r') as file : 
    reader = csv.reader(file) 
    rows = list(reader)[1:] 
    for row in rows : 
        range_start = int(row[0])
        range_end = int(row[1])
        
        start = time.time() 
        kv.range_query(range_start, range_end)
        end = time.time()
        kv_time = end - start
        
        start = time.time() 
        dict_kv.range_query(range_start, range_end)
        end = time.time()
        dict_time = end - start

        time_ratios.append(dict_time/kv_time)

In [26]:
# Visualization of time_ratios 

import matplotlib.pyplot as plt
plt.plot(time_ratios)
plt.xlabel('Query range result size')
plt.ylabel('Runtime ratio')
plt.show()

AttributeError: 'list' object has no attribute 'tolist'

We can see that in small query range, the speed of KVStore is faster 50 times than DictKVStore. Although the speed of KVStores is slower as query range wider, KVStore is faster than DictKVStore. 